# 환경 확인

In [15]:
import sys, os
# 1) 현재 파이썬 실행 파일 경로
print("Python executable:", sys.executable)

# 2) (conda 사용 시) 현재 활성화된 환경 이름
print("Conda env name:", os.environ.get("CONDA_DEFAULT_ENV"))

# 3) 패키지 검색 경로 확인
print("sys.path:")
for p in sys.path:
    print(" ", p)

Python executable: /home/jaecheon/miniconda3/envs/scenvi/bin/python
Conda env name: scenvi
sys.path:
  /home/jaecheon/miniconda3/envs/scenvi/lib/python310.zip
  /home/jaecheon/miniconda3/envs/scenvi/lib/python3.10
  /home/jaecheon/miniconda3/envs/scenvi/lib/python3.10/lib-dynload
  
  /home/jaecheon/.local/lib/python3.10/site-packages
  /home/jaecheon/miniconda3/envs/scenvi/lib/python3.10/site-packages
  /data/kjc1/mylit/mypy


# import

In [50]:
%load_ext autoreload
%autoreload 2
#모든 모듈을 import 할 때마다 자동으로 다시 불러오기
import mypy
import scanpy as sc
import pandas as pd
import numpy as np
from mypy.XGBoost import find_prognostic_genes_xgb, find_robust_prognostic_genes_with_downsampling, xgb_gset_finder

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
import sys
# mypkg 와 그 하위 모듈 모두 삭제
for mod in list(sys.modules):
    if mod.startswith("mypy"):
        del sys.modules[mod]

In [31]:
pd.set_option('display.max_rows', None)

In [5]:
adata=sc.read_h5ad("/data/kjc1/projects/#130.stroke/sobj/h5ad/is.h5ad")

In [6]:
adata

AnnData object with n_obs × n_vars = 33878 × 27285
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'Barcode', 'Best_Sample', 'Best_Probability', 'Second_Best_Sample', 'Second_Best_Probability', 'Probability_Ratio', 'droplet_demulti', 'GEM', 'join_key', 'day', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.8', 'seurat_clusters', 'droplet', 'sample_hto', 'GEM_hto', 'ol_time_point', 'sample_no', 'hos_no', 'sample_type', 'experiment_number', 'name', 'sex', 'birthdate', 'age', 'type', 'adm_date', 'sx_onset', 'blood_sample_time', 'csf_yn', 'csf_sample_time', 'csf_from', 'sample_done_time', 'exp_sample_no', 'set', 'multi_method', 'hto_no', 'charge', 'crcs_no', 'hash_id', 'sample_no.1', 'male', 'ht', 'wt', 'last_normal_dt', 'symptom_onset_dt', 'arrival_dt', 'blood_sample_time.1', 'LNT_to_sample_h', 'FAT_to_sample_h', 'hx_pre_mrs', 'toast', 'ivt', 'iat', 'ia_angio', 'pre_evt_state', 'post_evt_state', 'succ_recan', 'iv_start', 'ia_start', 'ia_end', 'lnt_to_iat_h', 'iat_to_sample_h', 'sHT', 'H

In [10]:
?find_prognostic_genes_xgb

Signature:
find_prognostic_genes_xgb(
    adata: anndata._core.anndata.AnnData,
    sample_label_col: str,
    prognosis_factor_col: str,
    cell_identity_col: str = None,
    target_cell_identity: str = None,
    use_pseudobulk: bool = True,
    n_top_genes: int = 50,
    test_size: float = 0.2,
    random_state: int = 42,
) -> pandas.core.frame.DataFrame
Docstring:
XGBoost를 사용하여 예후 인자와 관련된 유전자를 찾고 중요도 순으로 나열합니다.

Args:
    adata (anndata.AnnData): 입력 AnnData 객체. adata.X는 유전자 발현 행렬이어야 합니다.
    sample_label_col (str): adata.obs에서 샘플/환자 식별자 컬럼 이름. 유사벌크 생성 시 사용됩니다.
    prognosis_factor_col (str): adata.obs에서 예후 인자 컬럼 이름.
                                 (수치형 또는 'high'/'low'와 같은 이진 범주형).
    cell_identity_col (str, optional): adata.obs에서 세포 식별 정보 컬럼 이름.
                                       특정 세포 유형만 분석 대상일 경우 사용. Defaults to None.
    target_cell_identity (str, optional): cell_identity_col에서 분석 대상이 되는 특정 세포 클러스터 레이블.
                                          cell_identity_col이 제공된 경우 필

In [19]:
genes_total=find_prognostic_genes_xgb(adata,"exp_sample_no","nih_change")
genes_total_=find_prognostic_genes_xgb(adata,"exp_sample_no","group3")
genes_in_tc4=find_prognostic_genes_xgb(adata, "exp_sample_no", "nih_change", "annotation2_big", "CD4_T_Cell", n_top_genes=200)
genes_in_tc4_=find_prognostic_genes_xgb(adata, "exp_sample_no", "group3", "annotation2_big", "CD4_T_Cell", n_top_genes=200)
gene_in_tc4_boot=find_robust_prognostic_genes_with_downsampling(adata,"exp_sample_no","nih_change",cell_identity_col="annotation2_big", target_cell_identity="CD4_T_Cell")
gene_in_tc4_boot_=find_robust_prognostic_genes_with_downsampling(adata,"exp_sample_no","group3",cell_identity_col="annotation2_big", target_cell_identity="CD4_T_Cell")

Filtering for cell identity: CD4_T_Cell
Prognosis factor is numeric. Treating as regression.
XGBoost Regressor MSE: 82.2047


In [32]:
print(genes_in_tc4)
print(genes_in_tc4_)
print(genes_total)
print(genes_total_)
print(gene_in_tc4_boot)
print(gene_in_tc4_boot_)

(               gene    importance
5418         NUDT12  2.419886e-01
9085          TOR1B  1.478620e-01
1670            CR2  1.450740e-01
121        RERE-AS1  8.928900e-02
875             AGL  8.804797e-02
1426          MPZL1  5.319317e-02
275             ID3  4.784174e-02
10147         UEVLD  4.595370e-02
136      PIK3CD-AS2  4.291345e-02
11483          CDK2  2.429947e-02
5            FAM41C  1.164970e-02
14553          IRF8  1.074016e-02
21          B3GALT6  8.892450e-03
372            RCC1  8.649710e-03
674            NDC1  5.779915e-03
115            UTS2  5.436709e-03
777      AC103591.3  3.574454e-03
3391      LINC02084  2.937950e-03
4         LINC00115  2.076031e-03
164           FBXO6  2.012528e-03
55             GNB1  1.720601e-03
8841     AL133477.2  1.639488e-03
1671            CR1  1.460504e-03
8            KLHL17  1.409509e-03
18         TNFRSF18  1.358297e-03
51         SLC35E2B  1.245507e-03
2         LINC01409  8.817185e-04
14210         ITGAX  5.072025e-04
2682       AM

In [51]:
gset=xgb_gset_finder(adata,"exp_sample_no","nih_change")

Signature:
xgb_gset_finder(
    adata: anndata._core.anndata.AnnData,
    gene_sets: dict,
    prognosis_factor_col: str,
    sample_label_col: str = None,
    use_pseudobulk: bool = True,
    cell_identity_col: str = None,
    target_cell_identity: str = None,
    n_top_gsets: int = 20,
    test_size: float = 0.2,
    random_state: int = 42,
    gsva_method: str = 'gsva',
    gsva_kcdf: str = 'Gaussian',
) -> (<class 'pandas.core.frame.DataFrame'>, <class 'dict'>)
Docstring:
GSVA 점수를 특성으로 사용하여 XGBoost로 예후 관련 유전자 세트를 찾습니다.

Args:
    adata (anndata.AnnData): 입력 AnnData. adata.X는 정규화된 발현량 권장.
    gene_sets (dict): 유전자 세트 딕셔너리.
    prognosis_factor_col (str): 예후 인자 컬럼.
    sample_label_col (str): 유사벌크 시 샘플 ID 컬럼.
    use_pseudobulk (bool): 유사벌크 사용 여부.
    cell_identity_col (str, optional): 세포 식별 정보 컬럼.
    target_cell_identity (str, optional): 분석 대상 세포 클러스터.
    n_top_gsets (int): 반환할 상위 유전자 세트 개수.
    test_size (float): 테스트셋 비율.
    random_state (int): 난수 시드.
    gsva_method (str): GSVA

# GSEA

In [45]:
import pandas as pd
import gseapy as gp

# 1. XGBoost 결과로 얻은 유전자 중요도 DataFrame (가정)
#    results_df는 'gene'과 'importance' 컬럼을 가져야 함
#    results_df = xgb_gene_finder(...) # 이전 단계에서 실행

# 예시 데이터 (실제로는 XGBoost 결과 사용)

results_df=genes_total

# GSEApy prerank 모듈은 유전자 이름과 랭크 메트릭(여기서는 중요도)을 요구
# Series 또는 두 컬럼짜리 DataFrame으로 전달 가능
ranked_genes = results_df[['gene', 'importance']]

# 2. 유전자 세트 파일 (예: MSigDB의 hallmark gene sets GMT 파일)
# 실제 경로로 변경해야 합니다.
gene_sets_gmt = 'h.all.v7.4.symbols.gmt' # 예시 GMT 파일

# 3. GSEApy Prerank 실행
#    permutation_num은 보통 1000 이상으로 설정 (시간이 오래 걸릴 수 있음)
try:
    pre_res = gp.prerank(rnk=ranked_genes,
                         gene_sets=gene_sets_gmt,
                         permutation_num=100, # 예시를 위해 줄임, 실제로는 1000+
                         outdir='gsea_prerank_output', # 결과 저장 폴더
                         format='png',
                         seed=42)

    print("GSEA Prerank 결과 요약:")
    print(pre_res.res2d.head())

    # 특정 유전자 세트의 농축 플롯 보기 (결과 폴더에 저장됨)
    # terms = pre_res.res2d.index[:5] # 상위 5개 유전자 세트
    # for term in terms:
    #     gp.plot.plot_gsea(pre_res.res2d, term=term, ofname=f'gsea_plot_{term.replace(":", "_")}.png')

except Exception as e:
    print(f"GSEApy 실행 중 오류 발생: {e}")
    print("gseapy와 유전자 세트 파일이 올바르게 설치/지정되었는지 확인하세요.")
    print("예시 GMT 파일 'h.all.v7.4.symbols.gmt'은 Broad Institute MSigDB에서 다운로드 가능합니다.")

TypeError: tuple indices must be integers or slices, not list